# 機械学習に関するテクニック
機械学習でよく使うテクニックをまとめる。

In [1]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder
# # import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,cross_val_score, KFold, RepeatedKFold,StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer,KNNImputer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
# from sklearn import tree
# from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,AdaBoostClassifier, GradientBoostingRegressor, GradientBoostingClassifier


%matplotlib inline
import matplotlib.pyplot as plt


## ○ Polars

### - 欠損値を含むカラムの一覧を取得する

In [3]:
# データセットのカラム名やデータ型は分かっているので、dtypesを指定しておく。
dtypes = {
    "species": str,
    'island': str,
    'culmen_length_mm': pl.Float32, # くちばしの長さ[mm]
    'culmen_depth_mm': pl.Float32, # くちばしの高さ[mm]
    'flipper_length_mm': pl.Float32, # 翼の長さ[mm]
    'body_mass_g': pl.Float32, # 体重[g]
    'sex': str
}

# ペンギンのデータセット読み込み。欠損値がNAとして含まれているので、null_values="NA"を指定しないと読み込みエラーになる。
df = pl.read_csv('../Python/sample_data/ML_sample/penguins_size.csv',dtypes=dtypes, null_values='NA')

#### 方法１

In [28]:
(df
 .null_count().transpose(include_header=True)
 .filter(pl.col('column_0') > 0)
 .get_column('column').to_list()
)

['culmen_length_mm',
 'culmen_depth_mm',
 'flipper_length_mm',
 'body_mass_g',
 'sex']

上記ではnull_countの結果をカラム名（ヘッダー）を含めて転置して、  
転置後の欠損値数(column_0)でフィルターし、その時のヘッダーを返している。

#### 方法2

In [5]:
# 欠損値を含むカラムの列インデックスを取得
null_counts = df.null_count().to_numpy().reshape(7)
# np.whereで欠損値が０以上のカラムインデックスを取得。np.whereの結果はタプルで返ってくるので注意。
null_contain_cols_idx = np.where(null_counts > 0)[0]

In [6]:
# インデックスに対応するカラム名を取得
null_contain_cols = np.array(df.columns)[null_contain_cols_idx]

少々煩雑だが、numpyでもできる。